In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTMClassifier(nn.Module):
    def __init__(self,input_dim, hidden_dim, output_dim, num_layers=1, fc_hidden_dim=64, head_dropout: float = 0.0):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )

        self.fc1 = nn.Linear(hidden_dim, fc_hidden_dim)
        self.fc2 = nn.Linear(fc_hidden_dim, output_dim)
        self.dropout = nn.Dropout(head_dropout)

    def forward(self, x):
        batch_size = x.size(0)
        device = x.device

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)

        out, (h_n, c_n) = self.lstm(x, (h0, c0))   # out: (B, T, H)

        # Take last time step
        feat = out[:, -1, :]                        # (B, H)

        # Two-layer head
        feat = F.relu(self.fc1(feat))
        feat = self.dropout(feat)
        logits = self.fc2(feat)                     # raw logits (B, C)

        return logits, (h_n, c_n)
        



In [15]:
import torch
import utils as utils
import os

evaluate_order = ["blackhole_var5_base", "disflooding_var5_base", "worstparent_var5_base", "localrepair_var5_base"]

current_directory = os.getcwd()
domains_path = current_directory + '/data/attack_data'

domains = utils.create_domains(domains_path)

train_domains_loader = {}
test_domains_loader = {}

         
domains = utils.create_domains(domains_path)

train_domains_loader = {}
test_domains_loader = {}

for key, files in domains.items():
    _, test_domains_loader[key] = utils.load_data(domains_path, key, files, window_size=10, step_size=3, batch_size=256)
    
print(test_domains_loader.keys())

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte

In [ ]:
for scenario in evaluate_order:
    print(f"Evaluating scenario: {scenario}")
    domains = utils.create_domains(domains_path)

    train_domains_loader = {}
    test_domains_loader = {}

    for key, files in domains.items():
        _, test_domains_loader[key] = utils.load_data(domains_path, key, files, window_size=args.window_size, step_size=args.step_size, batch_size=args.batch_size)
          
    # Add your evaluation code here
    model = LSTMClassifier(input_dim=140, hidden_dim=10, output_dim=2, num_layers=1, fc_hidden_dim=10, head_dropout=0.05)
    state_dict = torch.load(
    "/Users/souba636/Documents/IoT_attack_CL_IDS/models/exp_no_1_LSTM_WCL_random/best_model_after_blackhole_var5_base.pt",
    map_location=torch.device("cpu"), weights_only=True
    )
    model.load_state_dict(state_dict)
